# UES Algorithm

This Notebook focuses on the re-implementation of the UES query plan optimization algorithm.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections
import json

import numpy as np
import pandas as pd

from transform import mosp, db

In [3]:
with open("../simplicity-done-right/JOB-Queries/implicit/19c.sql", "r") as query_file:
    example_query = " ".join(line.strip() for line in query_file.readlines())
example_query

"SELECT COUNT(*) FROM aka_name AS an, char_name AS chn, cast_info AS ci, company_name AS cn, info_type AS it, movie_companies AS mc, movie_info AS mi, name AS n, role_type AS rt, title AS t WHERE ci.note IN ('(voice)', '(voice: Japanese version)', '(voice) (uncredited)', '(voice: English version)') AND cn.country_code ='[us]' AND it.info = 'release dates' AND mi.info IS NOT NULL AND (mi.info LIKE 'Japan:%200%' OR mi.info LIKE 'USA:%200%') AND n.gender ='f' AND n.name LIKE '%An%' AND rt.role ='actress' AND t.production_year > 2000 AND t.id = mi.movie_id AND t.id = mc.movie_id AND t.id = ci.movie_id AND mc.movie_id = ci.movie_id AND mc.movie_id = mi.movie_id AND mi.movie_id = ci.movie_id AND cn.id = mc.company_id AND it.id = mi.info_type_id AND n.id = ci.person_id AND rt.id = ci.role_id AND n.id = an.person_id AND ci.person_id = an.person_id AND chn.id = ci.person_role_id; "

In [4]:
q = mosp.MospQuery.parse(example_query)
q.query

{'select': {'value': {'count': '*'}},
 'from': [{'value': 'aka_name', 'name': 'an'},
  {'value': 'char_name', 'name': 'chn'},
  {'value': 'cast_info', 'name': 'ci'},
  {'value': 'company_name', 'name': 'cn'},
  {'value': 'info_type', 'name': 'it'},
  {'value': 'movie_companies', 'name': 'mc'},
  {'value': 'movie_info', 'name': 'mi'},
  {'value': 'name', 'name': 'n'},
  {'value': 'role_type', 'name': 'rt'},
  {'value': 'title', 'name': 't'}],
 'where': {'and': [{'in': ['ci.note',
     {'literal': ['(voice)',
       '(voice: Japanese version)',
       '(voice) (uncredited)',
       '(voice: English version)']}]},
   {'eq': ['cn.country_code', {'literal': '[us]'}]},
   {'eq': ['it.info', {'literal': 'release dates'}]},
   {'exists': 'mi.info'},
   {'or': [{'like': ['mi.info', {'literal': 'Japan:%200%'}]},
     {'like': ['mi.info', {'literal': 'USA:%200%'}]}]},
   {'eq': ['n.gender', {'literal': 'f'}]},
   {'like': ['n.name', {'literal': '%An%'}]},
   {'eq': ['rt.role', {'literal': 'actres

In [5]:
q.collect_tables()

[aka_name AS an,
 char_name AS chn,
 cast_info AS ci,
 company_name AS cn,
 info_type AS it,
 movie_companies AS mc,
 movie_info AS mi,
 name AS n,
 role_type AS rt,
 title AS t]

In [6]:
q.predicates()

[ci.note in ['(voice)', '(voice: Japanese version)', '(voice) (uncredited)', '(voice: English version)'],
 cn.country_code = '[us]',
 it.info = 'release dates',
 mi.info IS NOT NULL,
 mi.info LIKE 'Japan:%200%' OR mi.info LIKE 'USA:%200%',
 n.gender = 'f',
 n.name LIKE '%An%',
 rt.role = 'actress',
 t.production_year > 2000,
 t.id = mi.movie_id,
 t.id = mc.movie_id,
 t.id = ci.movie_id,
 mc.movie_id = ci.movie_id,
 mc.movie_id = mi.movie_id,
 mi.movie_id = ci.movie_id,
 cn.id = mc.company_id,
 it.id = mi.info_type_id,
 n.id = ci.person_id,
 rt.id = ci.role_id,
 n.id = an.person_id,
 ci.person_id = an.person_id,
 chn.id = ci.person_role_id]

In [7]:
filters = [pred for pred in q.predicates() if not pred.is_join_predicate()]
filters

[ci.note in ['(voice)', '(voice: Japanese version)', '(voice) (uncredited)', '(voice: English version)'],
 cn.country_code = '[us]',
 it.info = 'release dates',
 mi.info IS NOT NULL,
 mi.info LIKE 'Japan:%200%' OR mi.info LIKE 'USA:%200%',
 n.gender = 'f',
 n.name LIKE '%An%',
 rt.role = 'actress',
 t.production_year > 2000]

In [8]:
join_predicates = [pred for pred in q.predicates() if pred.is_join_predicate()]
join_predicates

[t.id = mi.movie_id,
 t.id = mc.movie_id,
 t.id = ci.movie_id,
 mc.movie_id = ci.movie_id,
 mc.movie_id = mi.movie_id,
 mi.movie_id = ci.movie_id,
 cn.id = mc.company_id,
 it.id = mi.info_type_id,
 n.id = ci.person_id,
 rt.id = ci.role_id,
 n.id = an.person_id,
 ci.person_id = an.person_id,
 chn.id = ci.person_role_id]

In [9]:
class TableJoin:
    def __init__(self, predicate: mosp.MospPredicate):
        if not predicate.is_join_predicate():
            raise ValueError("Not a join predicate: '{}'".format(predicate))
        first, second = predicate.parse_left_attribute(), predicate.parse_right_attribute()
        if second.table.qualifier() < first.table.qualifier():
            first, second = second, first
        self.first = first.table
        self.second = second.table
        self.first_attr = first
        self.second_attr = second
        self.predicate = predicate
        self.pk, self.fk = None, None
    
    def mark_pk_fk_join(self, pk: db.TableRef, fk: db.TableRef):
        if pk not in [self.first, self.second] or fk not in [self.first, self.second]:
            raise ValueError(f"pk {pk} or fk {fk} not in join {self}")
        self.pk = pk
        self.fk = fk
    
    def is_pk_fk_join(self) -> bool:
        return self.pk is not None
    
    def is_n_m_join(self) -> bool:
        return self.pk is None
    
    def __hash__(self):
        return hash(self.predicate)
    
    def __eq__(self, other: object):
        if not isinstance(other, TableJoin):
            return False
        return self.predicate == other.predicate
    
    def __repr__(self):
        return str(self)
    
    def __str__(self):
        if self.is_pk_fk_join():
            return f"JOIN PK {self.pk}, FK {self.fk} ON {self.predicate}"
        return f"JOIN {self.first}, {self.second} ON {self.predicate}"
    

In [10]:
joins = set([TableJoin(pred) for pred in join_predicates])
joins

{JOIN aka_name AS an, cast_info AS ci ON ci.person_id = an.person_id,
 JOIN aka_name AS an, name AS n ON n.id = an.person_id,
 JOIN cast_info AS ci, movie_companies AS mc ON mc.movie_id = ci.movie_id,
 JOIN cast_info AS ci, movie_info AS mi ON mi.movie_id = ci.movie_id,
 JOIN cast_info AS ci, name AS n ON n.id = ci.person_id,
 JOIN cast_info AS ci, role_type AS rt ON rt.id = ci.role_id,
 JOIN cast_info AS ci, title AS t ON t.id = ci.movie_id,
 JOIN char_name AS chn, cast_info AS ci ON chn.id = ci.person_role_id,
 JOIN company_name AS cn, movie_companies AS mc ON cn.id = mc.company_id,
 JOIN info_type AS it, movie_info AS mi ON it.id = mi.info_type_id,
 JOIN movie_companies AS mc, movie_info AS mi ON mc.movie_id = mi.movie_id,
 JOIN movie_companies AS mc, title AS t ON t.id = mc.movie_id,
 JOIN movie_info AS mi, title AS t ON t.id = mi.movie_id}

In [11]:
dbs = db.DBSchema.get_instance()
for join in joins:
    pk, fk = None, None
    if dbs.is_primary_key(join.first_attr):
        pk = join.first
    elif dbs.is_primary_key(join.second_attr):
        pk = join.second
        
    if dbs.has_secondary_idx_on(join.first_attr):
        fk = join.first
    elif dbs.has_secondary_idx_on(join.second_attr):
        fk = join.second
        
    if pk and fk:
        join.mark_pk_fk_join(pk, fk)

PK/FK joins:

In [12]:
[join for join in joins if join.is_pk_fk_join()]

[JOIN PK role_type AS rt, FK cast_info AS ci ON rt.id = ci.role_id,
 JOIN PK title AS t, FK movie_info AS mi ON t.id = mi.movie_id,
 JOIN PK title AS t, FK movie_companies AS mc ON t.id = mc.movie_id,
 JOIN PK info_type AS it, FK movie_info AS mi ON it.id = mi.info_type_id,
 JOIN PK char_name AS chn, FK cast_info AS ci ON chn.id = ci.person_role_id,
 JOIN PK name AS n, FK cast_info AS ci ON n.id = ci.person_id,
 JOIN PK name AS n, FK aka_name AS an ON n.id = an.person_id,
 JOIN PK title AS t, FK cast_info AS ci ON t.id = ci.movie_id,
 JOIN PK company_name AS cn, FK movie_companies AS mc ON cn.id = mc.company_id]

N:M joins:

In [13]:
[join for join in joins if not join.is_pk_fk_join()]

[JOIN movie_companies AS mc, movie_info AS mi ON mc.movie_id = mi.movie_id,
 JOIN aka_name AS an, cast_info AS ci ON ci.person_id = an.person_id,
 JOIN cast_info AS ci, movie_info AS mi ON mi.movie_id = ci.movie_id,
 JOIN cast_info AS ci, movie_companies AS mc ON mc.movie_id = ci.movie_id]

In [14]:
predicate_map = collections.defaultdict(list)
for filter_pred in filters:
    predicate_map[filter_pred.parse_left_attribute().table].append(filter_pred)
predicate_map

defaultdict(list,
            {cast_info AS ci: [ci.note in ['(voice)', '(voice: Japanese version)', '(voice) (uncredited)', '(voice: English version)']],
             company_name AS cn: [cn.country_code = '[us]'],
             info_type AS it: [it.info = 'release dates'],
             movie_info AS mi: [mi.info IS NOT NULL,
              mi.info LIKE 'Japan:%200%' OR mi.info LIKE 'USA:%200%'],
             name AS n: [n.gender = 'f', n.name LIKE '%An%'],
             role_type AS rt: [rt.role = 'actress'],
             title AS t: [t.production_year > 2000]})

In [15]:
predicate_map[db.TableRef("movie_info", "mi")][0].parse_left_attribute().table

movie_info AS mi

In [16]:
p = mosp.CompoundMospFilterPredicate.build_and_predicate(predicate_map[db.TableRef("movie_info", "mi")])
p

mi.info IS NOT NULL AND mi.info LIKE 'Japan:%200%' OR mi.info LIKE 'USA:%200%'

In [17]:
p.estimate_result_rows()

explain (format json) SELECT * FROM movie_info AS mi WHERE mi.info IS NOT NULL AND (mi.info LIKE 'Japan:%200%' OR mi.info LIKE 'USA:%200%')


671749

In [23]:
a = p.parse_left_attribute()
a

mi.info

In [66]:
dbs = db.DBSchema.get_instance()
cur = dbs.cursor

In [26]:
cur.execute("select tablename, attname, most_common_vals, most_common_freqs from pg_stats where tablename = %s and attname = %s", (a.table.full_name, a.attribute))
rs = cur.fetchone()
tablename, attname, mcvs, mcfs = rs

In [73]:
dbs.load_most_common_values(a, k=3)

[('Color', 0.0649), ('English', 0.052733332), ('USA', 0.037766665)]

In [7]:
mosp.parse("SELECT * FROM foo TABLESAMPLE(bernoulli) WHERE a < 42")

{'select': '*',
 'from': {'value': 'foo', 'name': {'TABLESAMPLE': 'bernoulli'}},
 'where': {'lt': ['a', 42]}}

In [8]:
mosp.parse("SELECT * FROM foo f TABLESAMPLE(bernoulli) WHERE f.a < 42")

ParseException: Expecting {union} | {intersect} | {except} | {minus} | {order by} | {offset} | {fetch} | {limit} | {union} | {intersect} | {except} | {minus} | {StringEnd}, found "TABLESAMPL" (at char 20), (line:1, col:21)